DAT405 Assignment 4

Noah Lanai - 9808252192 - 12h work

Carl Hjalmarsson - 9305198930 - 12h work

In [2]:
import pandas as pd
import tarfile
from sklearn.model_selection import train_test_split
# Import libraries for Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

Task 1:

a, b, see code below

In [3]:
# Extract the email content, decode them, and convert as dataframe
def extract_files(files, m_class):  
    rows = []
    for fname in files:
        # open the tar file
        tfile = tarfile.open(fname, 'r:bz2')
        for member in tfile.getmembers():
            f = tfile.extractfile(member)
            if f is not None:
                row = f.read()
                #get all the content of file as a row
                rows.append({'message': row.decode('latin-1'), 'class': m_class})
        tfile.close()
    #return rows
    return pd.DataFrame(rows)

#get one dataframe with all of our files
df_ham = extract_files(['./20021010_easy_ham.tar.bz2','./20021010_hard_ham.tar.bz2'], 'ham')
df_spam = extract_files(['./20021010_spam.tar.bz2'], 'spam')

hamtrain, hamtest = train_test_split(df_ham, test_size = 0.25, random_state = 0)
spamtrain, spamtest = train_test_split(df_spam, test_size = 0.25, random_state = 0)

hamtrain.reset_index(drop=True, inplace=True)
hamtest.reset_index(drop=True, inplace=True)
spamtrain.reset_index(drop=True, inplace=True)
spamtest.reset_index(drop=True, inplace=True)

print(hamtrain)

                                                message class
0     From fork-admin@xent.com  Mon Aug 26 16:46:43 ...   ham
1     From rssfeeds@jmason.org  Tue Oct  1 10:37:39 ...   ham
2     From fork-admin@xent.com  Tue Sep 17 18:42:39 ...   ham
3     From spamassassin-talk-admin@lists.sourceforge...   ham
4     From fork-admin@xent.com  Tue Sep  3 14:24:41 ...   ham
...                                                 ...   ...
2095  From rpm-list-admin@freshrpms.net  Mon Sep  9 ...   ham
2096  From exmh-workers-admin@redhat.com  Wed Oct  2...   ham
2097  From rssfeeds@jmason.org  Tue Oct  1 10:36:45 ...   ham
2098  Return-Path: list-errors.1700002104.0.17199658...   ham
2099  Return-Path: Fool@motleyfool.com\nDelivery-Dat...   ham

[2100 rows x 2 columns]
